# DCF Modeling

In [1]:
import excelify as el

# Step 0: Assumptions

In [2]:
previous_tax_rates = [6858 / 20564, 4915 / 20116, 4281 / 11460]
assumptions_df = el.ExcelFrame(
    {
        "Company Name": ["Walmart Inc."],
        "Ticker": ["WMT"],
        "Current Share Price": [139.43],
        "Effective Tax Rate": [sum(previous_tax_rates) / len(previous_tax_rates)],
        "Last Fiscal Year": ["2021-01-31"],
    },
)
assumptions_df

Company Name,Ticker,Current Share Price,Effective Tax Rate,Last Fiscal Year
Walmart Inc.,WMT,139.43,0.31712950256223044,2021-01-31


## Step 1: Unlevered Free Cash Flow

In [3]:
def nopat(operating_income, taxes):
    return operating_income + taxes


def deferred_taxes(taxes, percent_book_taxes):
    return -taxes * percent_book_taxes


def unlevered_free_cash_flow(
    nopat, d_and_a, deferred_taxes, other_operating, change_in_working_capial, capex
):
    return (
        nopat
        + d_and_a
        + deferred_taxes
        + other_operating
        + change_in_working_capial
        + capex
    )

In [5]:
historical_ufcf_df = el.ExcelFrame(
    {
        "Year": ["2019", "2020", "2021"],
        "Net Sales": [510_329.0, 519_926.0, 555_233.0],
        "Membership & Other Income": [4076.0, 4038.0, 3918.0],
        "Operating Income (EBIT)": [21_957.0, 20_568.0, 22_548.0],
        "Depreciation & Amortization": [10_678.0, 10_987.0, 11_152.0],
        "% Book Taxes": [-499 / 4281, 320 / 4915, 1911 / 6858],
        "Other Operating Activities": [1_734, 1_981, 1_521],
        "Change in Working Capital": [295, -327, 7972],
        "Capital Expenditures": [-10_344, -10_705, -10_264],
    }
)

historical_ufcf_df = historical_ufcf_df.with_columns(
    (
        -el.col("Operating Income (EBIT)")
        * el.SingleCellExpr(assumptions_df["Effective Tax Rate"][0])
    ).alias("Taxes, Excluding Effect of Interest"),
)

historical_ufcf_df = historical_ufcf_df.with_columns(
    nopat(
        el.col("Operating Income (EBIT)"),
        el.col("Taxes, Excluding Effect of Interest"),
    ).alias("Net Operating Profit After Tax (NOPAT)"),
    deferred_taxes(
        el.col("Taxes, Excluding Effect of Interest"),
        el.col("% Book Taxes"),
    ).alias("Deferred Taxes"),
)

historical_ufcf_df = historical_ufcf_df.with_columns(
    unlevered_free_cash_flow(
        el.col("Net Operating Profit After Tax (NOPAT)"),
        el.col("Depreciation & Amortization"),
        el.col("Deferred Taxes"),
        el.col("Other Operating Activities"),
        el.col("Change in Working Capital"),
        el.col("Capital Expenditures"),
    ).alias("Annual Unlevered Free Cash Flow"),
)

# TODO: Allow displaying in transpose.
# TODO: Display fewer decimal points.
historical_ufcf_df.evaluate()

Year,Net Sales,Membership & Other Income,Operating Income (EBIT),Depreciation & Amortization,% Book Taxes,Other Operating Activities,Change in Working Capital,Capital Expenditures,"Taxes, Excluding Effect of Interest",Net Operating Profit After Tax (NOPAT),Deferred Taxes,Annual Unlevered Free Cash Flow
2019,510329.0,4076.0,21957.0,10678.0,-0.11656155103947675,1734,295,-10344,-6963.212487758894,14993.787512241106,-811.6428477906302,16545.144664450476
2020,519926.0,4038.0,20568.0,10987.0,0.06510681586978637,1981,-327,-10705,-6522.719608699956,14045.280391300044,424.673504533873,16405.953895833918
2021,555233.0,3918.0,22548.0,11152.0,0.2786526684164479,1521,7972,-10264,-7150.6360237731715,15397.363976226829,1992.5438088991732,27770.907785126
